# [Task 3: Multimodal Hate Speech Detection in Memes](https://github.com/marsadlab/MAHED2025Dataset/tree/main/task3/) at [ArabicNLP 2025](http://arabicnlp2025.sigarab.org/) @EMNLP 2025


Given multimodal content (text extracted from meme and the meme itself) the task is to detect whether the content is hateful or not-hateful. This is a binary classification task.

### installing required libraries.
 - transformers
 - datasets
 - evaluate
 - accelerate

In [11]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install --upgrade accelerate
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


#### importing required libraries and setting up logger

In [1]:
import logging
import os
import random
import sys
from dataclasses import dataclass, field
from typing import Optional
import pandas as pd
import datasets
import evaluate
import numpy as np
from torchvision.transforms import Compose, Normalize, ToTensor, Resize, CenterCrop
from datasets import load_dataset, Dataset, DatasetDict
import torch

import transformers
from transformers import (
    ConvNextFeatureExtractor,
    ResNetConfig,
    ResNetForImageClassification,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version


logger = logging.getLogger(__name__)

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)

### Setting up the training parameters

In [2]:
training_args = TrainingArguments(
    learning_rate=2e-5,
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    output_dir="./resnet_50/",
    overwrite_output_dir=True,
    remove_unused_columns=False,
    local_rank= 1,
    load_best_model_at_end=True,
    save_total_limit=2,
    save_strategy="no"
)

max_train_samples = None
max_eval_samples=None
max_predict_samples=None
batch_size = 16

In [3]:
transformers.utils.logging.set_verbosity_info()

log_level = training_args.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f" distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
logger.info(f"Training/evaluation parameters {training_args}")

INFO:__main__:Training/evaluation parameters TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=Interv

#### Defining the Model

In [4]:
model_name = 'resnet50'

#### setting the random seed

In [5]:
set_seed(training_args.seed)

## Download data from HF: https://huggingface.co/datasets/QCRI/Prop2Hate-Meme
### Defining the training, validation, and test data

In [38]:
from datasets import load_dataset

dataset = load_dataset("QCRI/Prop2Hate-Meme")

# Specify the directory where you want to save the dataset

output_dir="./Prop2Hate-Meme"

# Save the dataset to the specified directory. This will save all splits to the output directory.
dataset.save_to_disk(output_dir)

# If you want to get the raw images from HF dataset format

from PIL import Image
import os
import json

# Directory to save the images
output_dir="./Prop2Hate-Meme/"
os.makedirs(output_dir, exist_ok=True)

# Iterate over the dataset and save each image
for split in ['train','dev','test']:
    jsonl_path = os.path.join(output_dir, f"arabic_hateful_meme_{split}.jsonl")
    with open(jsonl_path, 'w', encoding='utf-8') as f:
        for idx, item in enumerate(dataset[split]):
            # Access the image directly as it's already a PIL.Image object
            image = item['image']
            image_path = os.path.join(output_dir, item['img_path'])
            # Ensure the directory exists
            os.makedirs(os.path.dirname(image_path), exist_ok=True)
            image.save(image_path)
            del item['image']
            del item['prop_label']
            del item['hate_fine_grained_label']
            item['label'] = item.pop('hate_label')
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

Found cached dataset prop2_hate-meme (/root/.cache/huggingface/datasets/QCRI___prop2_hate-meme/default/0.0.0/7315cbe87f45750b8b027c66fa662b536bce8248)
INFO:datasets.builder:Found cached dataset prop2_hate-meme (/root/.cache/huggingface/datasets/QCRI___prop2_hate-meme/default/0.0.0/7315cbe87f45750b8b027c66fa662b536bce8248)


Saving the dataset (0/1 shards):   0%|          | 0/2143 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/606 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/312 [00:00<?, ? examples/s]

In [63]:
import os
os.chdir("Prop2Hate-Meme")

train_file = './arabic_hateful_meme_train.jsonl'
validation_file = './arabic_hateful_meme_dev.jsonl'
test_file = './arabic_hateful_meme_test.jsonl'

In [64]:
jsonl_path = "./arabic_hateful_meme_train.jsonl" # Example path, modify as needed
data = []
with open(jsonl_path, 'r', encoding='utf-8') as f:
  for line in f:
    data.append(json.loads(line))

# data is now a list of dictionaries, where each dictionary is a parsed JSON object from a line in the file.
print(f"Loaded {len(data)} entries from {jsonl_path}")
if data:
    print("First entry:")
data[0]

Loaded 2143 entries from ./arabic_hateful_meme_train.jsonl
First entry:


{'id': 'data/arabic_memes_fb_insta_pinterest/Pinterest/images/pinterest_images_part2/www.pinterest.com_pin_374924737743995066/7485ad3c9c4cd8159ce93997a18a53a8.jpg',
 'text': 'زوجة ماكرون تصرح أن الحجاب يرعب ويخيف الأطفال..😅😂😂',
 'img_path': './data/arabic_memes_fb_insta_pinterest/Pinterest/images/pinterest_images_part2/www.pinterest.com_pin_374924737743995066/7485ad3c9c4cd8159ce93997a18a53a8.jpg',
 'label': 0}

#### Loading data files

In [80]:
import pandas as pd
from datasets import Dataset, DatasetDict, Image

def read_jsonl_to_df(filename):
    return pd.read_json(filename, lines=True)

l2id = {'not-hateful': 0, 'hateful': 1}

# Assume all splits use "img_path" as the image column
def prepare_dataset(file):
    df = read_jsonl_to_df(file)
    # df['label'] = df['label'].map(l2id)
    # Cast "img_path" column as Image
    return Dataset.from_pandas(df).cast_column("img_path", Image())

train_dataset = prepare_dataset(train_file)
validation_dataset = prepare_dataset(validation_file)
test_dataset = prepare_dataset(test_file)

raw_datasets = DatasetDict(
    {"train": train_dataset, "validation": validation_dataset, "test": test_dataset}
)



##### Extracting number of unique labels

In [83]:
# Labels
label_list = raw_datasets["train"].unique("label")
label_list.sort()  # sort the labels for determine
num_labels = len(label_list)
label_list

[0, 1]

### Loading Pretrained Configuration, Tokenizer and Model

In [84]:
config = ResNetConfig(
        num_channels=1,
        layer_type="basic",
        depths=[2, 2],
        hidden_sizes=[32, 64],
        num_labels=num_labels,
)

model = ResNetForImageClassification(config)

#### Preprocessing the raw_datasets

In [85]:
feature_extractor = ConvNextFeatureExtractor(
    do_resize=True, do_normalize=False, image_mean=[0.45], image_std=[0.22]
)
normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
_transforms = Compose([Resize(256), CenterCrop(224), ToTensor(), normalize])

def transforms(example_batch):
    """Apply _train_transforms across a batch."""
    # print(example_batch)
    # black and white
    example_batch["pixel_values"] = [_transforms(pil_img.convert("L")) for pil_img in example_batch["img_path"]]
    return example_batch

/usr/local/lib/python3.11/dist-packages/transformers/models/convnext/feature_extraction_convnext.py:30: FutureWarning: The class ConvNextFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ConvNextImageProcessor instead.
  warnings.warn(


#### Finalize the training data for training the model

In [86]:
if "train" not in raw_datasets:
    raise ValueError("requires a train dataset")
train_dataset = raw_datasets["train"]
if max_train_samples is not None:
    max_train_samples_n = min(len(train_dataset), max_train_samples)
    train_dataset = train_dataset.select(range(max_train_samples_n))
train_dataset.set_transform(transforms)

In [87]:
train_dataset

Dataset({
    features: ['id', 'text', 'img_path', 'label'],
    num_rows: 2143
})

#### Finalize the development/evaluation data for evaluating the model

In [127]:
if "validation" not in raw_datasets:
    raise ValueError("requires a validation dataset")
eval_dataset = raw_datasets["validation"]
if max_eval_samples is not None:
    max_eval_samples_n = min(len(eval_dataset), max_eval_samples)
    eval_dataset = eval_dataset.select(range(max_eval_samples_n))
eval_dataset.set_transform(transforms)
eval_dataset

Dataset({
    features: ['id', 'text', 'img_path', 'label'],
    num_rows: 312
})

#### Finalize the test data for predicting the unseen test data using the model

In [128]:
if "test" not in raw_datasets and "test_matched" not in raw_datasets:
    raise ValueError("requires a test dataset")
predict_dataset = raw_datasets["test"]
if max_predict_samples is not None:
    max_predict_samples_n = min(len(predict_dataset), max_predict_samples)
    predict_dataset = predict_dataset.select(range(max_predict_samples_n))
predict_dataset.set_transform(transforms)
predict_dataset

Dataset({
    features: ['id', 'text', 'img_path', 'label'],
    num_rows: 606
})

#### Log a few random samples from the training set

In [113]:
for index in random.sample(range(len(train_dataset)), 3):
    logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

INFO:__main__:Sample 456 of the training set: {'id': 'data/arabic_memes_fb_insta_pinterest/Facebook/images/salsamemers/333730289_873615157259879_2315221019976095254_n.jpg', 'text': 'المسيخ الدجال جا يحكم اليمن. \nلشعب اليمني: \nشكرا دجال الخير\nمالنا الا المسيخ\nاضرب بنا البحر ياسيدي الاعور ', 'img_path': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=713x540 at 0x7DA0C3C97CD0>, 'label': 1, 'pixel_values': tensor([[[ 2.5000,  2.5000,  2.5000,  ...,  0.5927, -0.2451,  2.3217],
         [ 2.5000,  2.5000,  2.5000,  ...,  0.1649,  0.1827,  2.4109],
         [ 2.5000,  2.5000,  2.5000,  ..., -0.3877,  0.7353,  2.4109],
         ...,
         [ 0.3253,  0.2718,  0.2540,  ...,  1.5374,  1.0918,  0.6818],
         [ 0.2897,  0.2005,  0.1292,  ...,  1.1275, -0.2807, -0.4768],
         [ 0.1827,  0.0936, -0.0134,  ..., -0.4234,  0.6640, -0.0134]]])}.
INFO:__main__:Sample 102 of the training set: {'id': 'data/arabic_memes_fb_insta_pinterest/Instagram/IMAGES/ex.officiall/2020-07-08_15-01-

#### Get the metric function `accuracy`

In [91]:
metric = evaluate.load("accuracy")

#### Predictions and label_ids field and has to return a dictionary string to float.

In [114]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}


#### Data Collator

In [115]:
def collate_fn(examples):
    # print(examples)
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}
data_collator = collate_fn

#### Initialize our Trainer

In [116]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset, # if you have development and test set, uncomment this line
    compute_metrics=compute_metrics,
    tokenizer=feature_extractor,
    data_collator=data_collator,
)

/tmp/ipython-input-116-1272168917.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


#### Training our model

In [117]:
train_result = trainer.train()
metrics = train_result.metrics
max_train_samples = (
    max_train_samples if max_train_samples is not None else len(train_dataset)
)
metrics["train_samples"] = min(max_train_samples, len(train_dataset))



[INFO|trainer.py:2402] 2025-07-18 21:14:19,099 >> ***** Running training *****
[INFO|trainer.py:2403] 2025-07-18 21:14:19,100 >>   Num examples = 2,143
[INFO|trainer.py:2404] 2025-07-18 21:14:19,101 >>   Num Epochs = 2
[INFO|trainer.py:2405] 2025-07-18 21:14:19,102 >>   Instantaneous batch size per device = 16
[INFO|trainer.py:2408] 2025-07-18 21:14:19,103 >>   Total train batch size (w. parallel, distributed & accumulation) = 16
[INFO|trainer.py:2409] 2025-07-18 21:14:19,105 >>   Gradient Accumulation steps = 1
[INFO|trainer.py:2410] 2025-07-18 21:14:19,106 >>   Total optimization steps = 268
[INFO|trainer.py:2411] 2025-07-18 21:14:19,107 >>   Number of trainable parameters = 183,554
[INFO|integration_utils.py:832] 2025-07-18 21:14:19,113 >> Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss


[INFO|trainer.py:2677] 2025-07-18 21:14:55,962 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)




#### Saving the tokenizer too for easy upload

In [118]:
trainer.save_model()
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

[INFO|trainer.py:4019] 2025-07-18 21:14:55,977 >> Saving model checkpoint to ./resnet_50/
[INFO|configuration_utils.py:437] 2025-07-18 21:14:55,982 >> Configuration saved in ./resnet_50/config.json
[INFO|modeling_utils.py:3949] 2025-07-18 21:14:55,991 >> Model weights saved in ./resnet_50/model.safetensors
[INFO|image_processing_base.py:258] 2025-07-18 21:14:55,993 >> Image processor saved in ./resnet_50/preprocessor_config.json


***** train metrics *****
  epoch                    =        2.0
  total_flos               =   220578GF
  train_loss               =     0.3142
  train_runtime            = 0:00:36.85
  train_samples            =       2143
  train_samples_per_second =    116.288
  train_steps_per_second   =      7.271


#### Evaluating our model on validation/development data

In [119]:
logger.info("*** Evaluate ***")

metrics = trainer.evaluate(eval_dataset=eval_dataset)

max_eval_samples = (
    max_eval_samples if max_eval_samples is not None else len(eval_dataset)
)
metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))

trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

INFO:__main__:*** Evaluate ***
[INFO|trainer.py:4353] 2025-07-18 21:14:56,009 >> 
***** Running Evaluation *****
[INFO|trainer.py:4355] 2025-07-18 21:14:56,009 >>   Num examples = 312
[INFO|trainer.py:4358] 2025-07-18 21:14:56,011 >>   Batch size = 16


***** eval metrics *****
  epoch                   =        2.0
  eval_accuracy           =     0.9006
  eval_loss               =     0.3056
  eval_runtime            = 0:00:01.97
  eval_samples            =        312
  eval_samples_per_second =    158.221
  eval_steps_per_second   =     10.142


### Predecting the test data

In [129]:
# if the test set is available, you don't need to run this cell
# predict_dataset = eval_dataset


In [130]:
id2l = {0:'not-hateful', 1:'hateful'}
logger.info("*** Predict ***")

predictions = trainer.predict(predict_dataset, metric_key_prefix="predict").predictions
predictions = np.argmax(predictions, axis=1)
output_predict_file = os.path.join(training_args.output_dir, f"task2_TeamName.tsv")
if trainer.is_world_process_zero():
    with open(output_predict_file, "w") as writer:
        logger.info(f"***** Predict results *****")
        writer.write("id\tlabel\n")
        for index, item in enumerate(predictions):
            item = label_list[item]
            item = id2l[item]
            writer.write(f"{predict_dataset[index]['id']}\t{item}\n")

INFO:__main__:*** Predict ***
[INFO|trainer.py:4353] 2025-07-18 21:17:32,744 >> 
***** Running Prediction *****
[INFO|trainer.py:4355] 2025-07-18 21:17:32,745 >>   Num examples = 606
[INFO|trainer.py:4358] 2025-07-18 21:17:32,746 >>   Batch size = 16


INFO:__main__:***** Predict results *****


#### Saving the model into card

In [131]:
kwargs = {"finetuned_from": model_name, "tasks": "image-classification"}
trainer.create_model_card(**kwargs)

[INFO|modelcard.py:450] 2025-07-18 21:17:41,802 >> Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Image Classification', 'type': 'image-classification'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.9006410241127014}]}
